# Loading Dataset by Kagglehub

In [1]:
import kagglehub

path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

100%|██████████| 787M/787M [00:33<00:00, 24.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1


# Data Preparation and Splitting

In [2]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from PIL import UnidentifiedImageError

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
EPOCHS = 5
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2

DATASET_PATH = os.path.join(path, 'kagglecatsanddogs_3367a', 'PetImages')
SPLIT_DATASET_PATH = "PetImages_Split"
CLASSES = ["Cat", "Dog"]

# Function to clean and split the dataset
def clean_and_split_dataset(dataset_path, output_path, train_ratio=0.7, val_ratio=0.2):
    # Create train/val/test directories
    for subset in ["train", "val", "test"]:
        for cls in CLASSES:
            os.makedirs(os.path.join(output_path, subset, cls), exist_ok=True)

    # Process and split each class
    for cls in CLASSES:
        cls_path = os.path.join(dataset_path, cls)
        images = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # Check and clean corrupt images
        valid_images = []
        for img in images:
            img_path = os.path.join(cls_path, img)
            try:
                tf.keras.preprocessing.image.load_img(img_path).close()
                valid_images.append(img)
            except (UnidentifiedImageError, OSError):
                os.remove(img_path)
                print(f"Removed corrupt image: {img_path}")

        # Split into train, val, and test
        train, temp = train_test_split(valid_images, train_size=train_ratio, random_state=42)
        val, test = train_test_split(temp, test_size=(1 - train_ratio - val_ratio) / (1 - train_ratio), random_state=42)

        # Move files into respective folders
        for subset, split in zip(["train", "val", "test"], [train, val, test]):
            for img in split:
                src = os.path.join(cls_path, img)
                dst = os.path.join(output_path, subset, cls, img)
                shutil.copy(src, dst)

    print("Dataset cleaned and split into train/val/test subsets.")

# Clean and split the dataset
clean_and_split_dataset(DATASET_PATH, SPLIT_DATASET_PATH)



/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Dataset cleaned and split into train/val/test subsets.


## Data Loading and Augmentation

In [3]:
# Data generators
data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training and validation datasets
train_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "train"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

validation_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "val"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Test dataset
test_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "test"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

Found 17471 images belonging to 2 classes.
Found 4990 images belonging to 2 classes.
Found 2498 images belonging to 2 classes.


# Training Base Model (MobileNet) on "Cat vs Dog" dataset and checking results

In this step, we define a MobileNet-based CNN architecture from scratch (no pre-trained weights), compile it with the Adam optimizer, and train it on the Cat vs Dog dataset.
After training, we evaluate the model on the test set to check its performance in terms of loss and accuracy.

In [4]:
# Function to create MobileNet model for retraining
def create_model():
    # Initialize MobileNet with random weights
    base_mobilenet = tf.keras.applications.MobileNet(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        # No pre-trained weights
        weights=None
    )
    model = Sequential([
        base_mobilenet,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        # Binary classification output
        Dense(1, activation="sigmoid")
    ])
    return model

# Compile and train model
model = create_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

print("Retraining the MobileNet model from scratch...")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate model
loss, accuracy = model.evaluate(test_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Retraining the MobileNet model from scratch...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 153s 767ms/step - accuracy: 0.5826 - loss: 0.7050 - val_accuracy: 0.4996 - val_loss: 0.6932
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 455ms/step - accuracy: 0.6647 - loss: 0.6172 - val_accuracy: 0.4996 - val_loss: 0.7057
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 62s 448ms/step - accuracy: 0.7299 - loss: 0.5296 - val_accuracy: 0.4996 - val_loss: 0.7008
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 62s 452ms/step - accuracy: 0.7850 - loss: 0.4528 - val_accuracy: 0.5004 - val_loss: 0.8302
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 62s 453ms/step - accuracy: 0.8225 - loss: 0.3924 - val_accuracy: 0.7541 - val_loss: 0.4955
20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 798ms/step - accuracy: 0.7518 - loss: 0.5012
Validation Loss: 0.5045
Validation Accuracy: 0.7478


# Transfer Learning with MobileNet

In this step, we use MobileNet pre-trained on ImageNet as the base model and freeze its layers.
We then add custom classification layers on top and train only those layers on our Cat vs Dog dataset.

Finally, we evaluate the model on the test set to check its accuracy and loss.

In [5]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

def create_transfer_learning_model():
    # Load the MobileNet base model
    base_mobilenet = MobileNet(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    # Freeze the base layers
    base_mobilenet.trainable = False

    # Create a new model by adding custom layers on top of the pre-trained base
    model = Sequential([
        base_mobilenet,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile and train transfer learning model
transfer_learning_model = create_transfer_learning_model()
transfer_learning_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training Transfer Learning Model (MobileNet with ImageNet weights)...")
history_transfer = transfer_learning_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)


transfer_learning_model.save('transfer_learning_mobilenet_model.keras')

#MODEL EVALUATION

transfer_learning_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss, accuracy = transfer_learning_model.evaluate(test_generator)


print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training Transfer Learning Model (MobileNet with ImageNet weights)...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 68s 460ms/step - accuracy: 0.7709 - loss: 0.4673 - val_accuracy: 0.9772 - val_loss: 0.0676
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 55s 401ms/step - accuracy: 0.9680 - loss: 0.0898 - val_accuracy: 0.9828 - val_loss: 0.0502
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 54s 395ms/step - accuracy: 0.9747 - loss: 0.0656 - val_accuracy: 0.9840 - val_loss: 0.0433
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 460ms/step - accuracy: 0.9785 - loss: 0.0621 - val_accuracy: 0.9842 - val_loss: 0.0402
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 54s 396ms/step - accuracy: 0.9792 - loss: 0.0557 - val_accuracy: 0.9842 - val_loss: 0.0380
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 401ms/step - accuracy: 0.9849 - loss: 0.0444
Test Loss: 0.0449
Test Accuracy: 0.9864


# **Evaluating Different CNN Models**
## Here we train and evaluate four popular CNN architectures on the Cat vs Dog dataset to compare performance:
## 1. LeNet
## 2. AlexNet
## 3. VGGNet
## 4. ResNet

# 1. LeNet Model
We implement the LeNet architecture, one of the earliest CNNs, and train it on the Cat vs Dog dataset.
After training, we evaluate its performance on the test set.

In [6]:
##---------LeNet Model--------##
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_lenet_model():
    model = Sequential([
        Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(*IMAGE_SIZE, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, kernel_size=(5, 5), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

lenet_model = create_lenet_model()
lenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training LeNet Model...")
lenet_history = lenet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

lenet_model.save('lenet_model.keras')

# Evaluate LeNet model
lenet_loss, lenet_accuracy = lenet_model.evaluate(test_generator)
print(f"LeNet Test Loss: {lenet_loss:.4f}")
print(f"LeNet Test Accuracy: {lenet_accuracy:.4f}")


Training LeNet Model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 73s 476ms/step - accuracy: 0.5860 - loss: 0.6772 - val_accuracy: 0.7295 - val_loss: 0.5338
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 53s 387ms/step - accuracy: 0.7796 - loss: 0.4709 - val_accuracy: 0.7613 - val_loss: 0.4954
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 54s 398ms/step - accuracy: 0.8463 - loss: 0.3486 - val_accuracy: 0.7641 - val_loss: 0.5175
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 53s 388ms/step - accuracy: 0.9045 - loss: 0.2351 - val_accuracy: 0.7659 - val_loss: 0.6016
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 53s 388ms/step - accuracy: 0.9607 - loss: 0.1188 - val_accuracy: 0.7487 - val_loss: 0.7516
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 358ms/step - accuracy: 0.7468 - loss: 0.8244
LeNet Test Loss: 0.7846
LeNet Test Accuracy: 0.7526


# 2. AlexNet Model

We implement the AlexNet architecture, a deeper CNN that popularized deep learning in computer vision.
The model is trained on the Cat vs Dog dataset and evaluated on the test set.

In [7]:
##------------------AlexNet Model------------##
from tensorflow.keras.layers import BatchNormalization

def create_alexnet_model():
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=(*IMAGE_SIZE, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

alexnet_model = create_alexnet_model()
alexnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training AlexNet Model...")
alexnet_history = alexnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

alexnet_model.save('alexnet_model.keras')

# Evaluate AlexNet model
alexnet_loss, alexnet_accuracy = alexnet_model.evaluate(test_generator)
print(f"AlexNet Test Loss: {alexnet_loss:.4f}")
print(f"AlexNet Test Accuracy: {alexnet_accuracy:.4f}")


Training AlexNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 92s 537ms/step - accuracy: 0.5085 - loss: 6.0655 - val_accuracy: 0.5004 - val_loss: 0.6934
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 107s 403ms/step - accuracy: 0.4954 - loss: 0.6934 - val_accuracy: 0.5004 - val_loss: 0.6931
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 55s 399ms/step - accuracy: 0.4942 - loss: 0.6932 - val_accuracy: 0.5004 - val_loss: 0.6932
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 56s 409ms/step - accuracy: 0.5008 - loss: 0.6932 - val_accuracy: 0.5004 - val_loss: 0.6932
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 80s 395ms/step - accuracy: 0.4927 - loss: 0.6933 - val_accuracy: 0.5004 - val_loss: 0.6932
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 476ms/step - accuracy: 0.5091 - loss: 0.6931
AlexNet Test Loss: 0.6932
AlexNet Test Accuracy: 0.5004


# 3. VGGNet Model

We train the VGGNet architecture, which uses deeper layers with small filters for improved accuracy.

After training, its performance is tested on the Cat vs Dog dataset.

In [8]:
##-------------VGGNet Model--------------##
from tensorflow.keras.applications import VGG16

def create_vgg_model():
    # Load pre-trained VGG16 base model
    base_vgg = VGG16(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    # Freeze the base layers
    base_vgg.trainable = False

    # Add custom layers for transfer learning
    model = Sequential([
        base_vgg,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

vgg_model = create_vgg_model()
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training VGGNet Model...")
vgg_history = vgg_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

vgg_model.save('vggnet_model.keras')

# Evaluate VGGNet model
vgg_loss, vgg_accuracy = vgg_model.evaluate(test_generator)
print(f"VGGNet Test Loss: {vgg_loss:.4f}")
print(f"VGGNet Test Accuracy: {vgg_accuracy:.4f}")


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training VGGNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.5659 - loss: 0.7136 - val_accuracy: 0.9022 - val_loss: 0.4407
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 119s 872ms/step - accuracy: 0.8506 - loss: 0.4340 - val_accuracy: 0.9160 - val_loss: 0.3397
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 102s 741ms/step - accuracy: 0.8800 - loss: 0.3550 - val_accuracy: 0.9259 - val_loss: 0.2888
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 739ms/step - accuracy: 0.8891 - loss: 0.3158 - val_accuracy: 0.9305 - val_loss: 0.2577
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 740ms/step - accuracy: 0.8995 - loss: 0.2860 - val_accuracy: 0.9341 - val_loss: 0.2364
20/20 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.9430 - loss: 0.2279
VGGNet Test Loss: 0.2335
VGGNet Test Accuracy: 0.9400


# 4. ResNet Model

We implement the ResNet architecture, which introduces residual connections to train very deep networks.

The model is trained on the Cat vs Dog dataset and evaluated on the test set.

In [9]:
##----------ResNet Model---------------##
from tensorflow.keras.applications import ResNet50

def create_resnet_model():
    # Load pre-trained ResNet50 base model
    base_resnet = ResNet50(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    # Freeze the base layers
    base_resnet.trainable = False

    # Add custom layers for transfer learning
    model = Sequential([
        base_resnet,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

resnet_model = create_resnet_model()
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training ResNet Model...")
resnet_history = resnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

resnet_model.save('resnet_model.keras')

# Evaluate ResNet model
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_generator)
print(f"ResNet Test Loss: {resnet_loss:.4f}")
print(f"ResNet Test Accuracy: {resnet_accuracy:.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Training ResNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 107s 642ms/step - accuracy: 0.5627 - loss: 0.6993 - val_accuracy: 0.6435 - val_loss: 0.6358
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 64s 465ms/step - accuracy: 0.6187 - loss: 0.6420 - val_accuracy: 0.6549 - val_loss: 0.6254
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 460ms/step - accuracy: 0.6419 - loss: 0.6304 - val_accuracy: 0.6655 - val_loss: 0.6171
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 64s 466ms/step - accuracy: 0.6476 - loss: 0.6230 - val_accuracy: 0.6683 - val_loss: 0.6113
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 462ms/step - accuracy: 0.6534 - loss: 0.6204 - val_accuracy: 0.6629 - val_loss: 0.6125
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 708ms/step - accuracy: 0.6464 - loss: 0.6203
ResNet Test Loss: 0.6213
ResNet Test Accuracy: 0.6553


# Performing Transfer Learning using the following models and check results
# 1. LeNet
# 2. AlexNet
# 3. VGGNet
# 4. ResNet

# 1. LeNet Transfer Learning


In [10]:
##----------LeNet Transfer Learning-------------##
lenet_model = create_lenet_model()
lenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training LeNet Model...")
lenet_history = lenet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate LeNet Model
lenet_loss, lenet_accuracy = lenet_model.evaluate(test_generator)
print(f"LeNet Test Loss: {lenet_loss:.4f}")
print(f"LeNet Test Accuracy: {lenet_accuracy:.4f}")


Training LeNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 58s 403ms/step - accuracy: 0.5302 - loss: 0.7907 - val_accuracy: 0.6661 - val_loss: 0.6016
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 54s 395ms/step - accuracy: 0.7278 - loss: 0.5361 - val_accuracy: 0.7174 - val_loss: 0.5742
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 54s 391ms/step - accuracy: 0.8081 - loss: 0.4159 - val_accuracy: 0.7401 - val_loss: 0.5382
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 397ms/step - accuracy: 0.8834 - loss: 0.2855 - val_accuracy: 0.7427 - val_loss: 0.6162
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 81s 387ms/step - accuracy: 0.9429 - loss: 0.1588 - val_accuracy: 0.7423 - val_loss: 0.7650
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 319ms/step - accuracy: 0.7264 - loss: 0.7875
LeNet Test Loss: 0.8063
LeNet Test Accuracy: 0.7226


# 2. AlexNet Transfer Learning


In [11]:
##-------------------AlexNet Transfer Learning-------------##
alexnet_model = create_alexnet_model()
alexnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training AlexNet Model...")
alexnet_history = alexnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate AlexNet Model
alexnet_loss, alexnet_accuracy = alexnet_model.evaluate(test_generator)
print(f"AlexNet Test Loss: {alexnet_loss:.4f}")
print(f"AlexNet Test Accuracy: {alexnet_accuracy:.4f}")


Training AlexNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 448ms/step - accuracy: 0.5357 - loss: 6.2273 - val_accuracy: 0.5635 - val_loss: 0.7132
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 56s 405ms/step - accuracy: 0.5958 - loss: 0.6719 - val_accuracy: 0.6092 - val_loss: 0.6666
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 57s 415ms/step - accuracy: 0.6452 - loss: 0.6266 - val_accuracy: 0.6707 - val_loss: 0.6015
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 64s 470ms/step - accuracy: 0.6898 - loss: 0.5761 - val_accuracy: 0.7162 - val_loss: 0.5493
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 55s 401ms/step - accuracy: 0.7418 - loss: 0.5252 - val_accuracy: 0.6882 - val_loss: 0.5757
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 317ms/step - accuracy: 0.6621 - loss: 0.5813
AlexNet Test Loss: 0.5832
AlexNet Test Accuracy: 0.6693


# 3. VGGNet Transfer Learning


In [12]:
##-----------------VGGNet Transfer Learning--------------##
vgg_model = create_vgg_model()
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training VGGNet Model...")
vgg_history = vgg_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate VGGNet Model
vgg_loss, vgg_accuracy = vgg_model.evaluate(test_generator)
print(f"VGGNet Test Loss: {vgg_loss:.4f}")
print(f"VGGNet Test Accuracy: {vgg_accuracy:.4f}")


Training VGGNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 108s 765ms/step - accuracy: 0.6545 - loss: 0.6228 - val_accuracy: 0.8932 - val_loss: 0.4257
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 740ms/step - accuracy: 0.8463 - loss: 0.4188 - val_accuracy: 0.9136 - val_loss: 0.3330
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 740ms/step - accuracy: 0.8721 - loss: 0.3535 - val_accuracy: 0.9255 - val_loss: 0.2842
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 740ms/step - accuracy: 0.8907 - loss: 0.3074 - val_accuracy: 0.9295 - val_loss: 0.2541
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 101s 740ms/step - accuracy: 0.8960 - loss: 0.2863 - val_accuracy: 0.9335 - val_loss: 0.2332
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 580ms/step - accuracy: 0.9431 - loss: 0.2241
VGGNet Test Loss: 0.2305
VGGNet Test Accuracy: 0.9380


# 4. ResNet Transfer Learning


In [13]:
##-------------------ResNet Transfer Learning------------------##
resnet_model = create_resnet_model()
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training ResNet Model...")
resnet_history = resnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate ResNet Model
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_generator)
print(f"ResNet Test Loss: {resnet_loss:.4f}")
print(f"ResNet Test Accuracy: {resnet_accuracy:.4f}")


Training ResNet Model...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 531ms/step - accuracy: 0.5403 - loss: 0.7417 - val_accuracy: 0.6395 - val_loss: 0.6388
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 62s 455ms/step - accuracy: 0.6226 - loss: 0.6443 - val_accuracy: 0.6575 - val_loss: 0.6250
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 460ms/step - accuracy: 0.6439 - loss: 0.6289 - val_accuracy: 0.6641 - val_loss: 0.6171
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 63s 461ms/step - accuracy: 0.6540 - loss: 0.6220 - val_accuracy: 0.6699 - val_loss: 0.6145
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 81s 458ms/step - accuracy: 0.6604 - loss: 0.6168 - val_accuracy: 0.6792 - val_loss: 0.6070
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 449ms/step - accuracy: 0.6740 - loss: 0.6085
ResNet Test Loss: 0.6143
ResNet Test Accuracy: 0.6681


# Model Performance Comparison

In this step, we compare the test loss and accuracy of all implemented models (LeNet, AlexNet, VGGNet, and ResNet) on the Cat vs Dog dataset to evaluate which architecture performs best.

In [14]:
##-------------Compare Results-------------##
print("Comparison of Model Performance:")
print(f"LeNet   - Test Loss: {lenet_loss:.4f}, Test Accuracy: {lenet_accuracy:.4f}")
print(f"AlexNet - Test Loss: {alexnet_loss:.4f}, Test Accuracy: {alexnet_accuracy:.4f}")
print(f"VGGNet  - Test Loss: {vgg_loss:.4f}, Test Accuracy: {vgg_accuracy:.4f}")
print(f"ResNet  - Test Loss: {resnet_loss:.4f}, Test Accuracy: {resnet_accuracy:.4f}")


Comparison of Model Performance:
LeNet   - Test Loss: 0.8063, Test Accuracy: 0.7226
AlexNet - Test Loss: 0.5832, Test Accuracy: 0.6693
VGGNet  - Test Loss: 0.2305, Test Accuracy: 0.9380
ResNet  - Test Loss: 0.6143, Test Accuracy: 0.6681


# **Final Comparison Verdict**  

Among all tested models, **VGGNet achieved the best performance** with the highest accuracy (≈93.8%) and lowest test loss, making it the most effective architecture for the Cat vs Dog dataset.  
**LeNet** performed moderately well (≈75% accuracy), while **ResNet underperformed** in this setup (≈66%), and **AlexNet struggled** to generalize (≈50%).  
Overall, **VGGNet proves to be the most reliable model** for this task.
